### Main funtion that takes in two matrics (3D and 2D) and return calculated params


For starting create a random matrix

In [1]:
import random 

Specify number of observation points 

In [2]:
n=6

Create 3d matrix which represents world coordinate system and 2d matrix which represents camera coordinate system
Note: We'll append 1 in world coordinates so dim will be (nx4) for 2d it will be (nx2)

In [55]:
W = np.array([[random.randint(1,10) for i in range(3)]+[1] for j in range(n)])
W

array([[ 1,  5,  2,  1],
       [ 6,  8,  4,  1],
       [ 5,  9,  6,  1],
       [ 2,  5,  7,  1],
       [ 2, 10, 10,  1],
       [ 2,  1,  1,  1]])

In [56]:
C = np.array([[random.randint(200,400) for i in range(2)] for j in range(n)])
C

array([[243, 327],
       [271, 311],
       [385, 378],
       [201, 222],
       [390, 304],
       [304, 278]])

Now we will make the P matrix (from P.m = 0) which has dim (2nX12)

In [25]:
import numpy as np

In [68]:
P = np.zeros((2*n,12))
P

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

Now fill in the value according to the eqaution:    
Pi x m1 + 0 x m2 -  X0 x Pi x m3 = 0    
0 x m1 + Pi x m2 -  Y0 x Pi x m3 = 0

In [69]:
for i in range(n):
    P[i*2,0:4] = W[i]
    P[i*2,8:12] = -C[i][0]*W[i]
    P[i*2+1,4:8] = W[i]
    P[i*2+1,8:12] = -C[i][1]*W[i]
P

array([[ 1.000e+00,  5.000e+00,  2.000e+00,  1.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00, -2.430e+02, -1.215e+03,
        -4.860e+02, -2.430e+02],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  1.000e+00,
         5.000e+00,  2.000e+00,  1.000e+00, -3.270e+02, -1.635e+03,
        -6.540e+02, -3.270e+02],
       [ 6.000e+00,  8.000e+00,  4.000e+00,  1.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00, -1.626e+03, -2.168e+03,
        -1.084e+03, -2.710e+02],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  6.000e+00,
         8.000e+00,  4.000e+00,  1.000e+00, -1.866e+03, -2.488e+03,
        -1.244e+03, -3.110e+02],
       [ 5.000e+00,  9.000e+00,  6.000e+00,  1.000e+00,  0.000e+00,
         0.000e+00,  0.000e+00,  0.000e+00, -1.925e+03, -3.465e+03,
        -2.310e+03, -3.850e+02],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  5.000e+00,
         9.000e+00,  6.000e+00,  1.000e+00, -1.890e+03, -3.402e+03,
        -2.268e+03,

Now we will go on calculating SVD for Pt x P  matrix. Then choosing the Eigenvector corresponding to minimum eigen value to obtain the solution P x m  = 0  

In [78]:
u, s, vh = np.linalg.svd(P.T*P)
s.shape,vh.shape

((12,), (12, 12))

In [81]:
m = vh[np.argmin(s)]
m.shape,m

((12,),
 array([-3.74737459e-01,  4.72091052e-02,  2.53972456e-02, -4.59600292e-01,
        -4.76151556e-02,  8.94968852e-04, -3.13399828e-02, -8.01380424e-01,
        -2.62634254e-04, -9.13053192e-05,  1.59884975e-04, -2.19162991e-03]))

In [82]:
m = m.reshape((3,4))
m.shape,m

((3, 4), array([[-3.74737459e-01,  4.72091052e-02,  2.53972456e-02,
         -4.59600292e-01],
        [-4.76151556e-02,  8.94968852e-04, -3.13399828e-02,
         -8.01380424e-01],
        [-2.62634254e-04, -9.13053192e-05,  1.59884975e-04,
         -2.19162991e-03]]))

In [92]:
a1 = m[0,0:3]
a2 = m[1,0:3]
a3 = m[2,0:3]
B = m[:,-1]
a1,a2,a3,B

(array([-0.37473746,  0.04720911,  0.02539725]),
 array([-0.04761516,  0.00089497, -0.03133998]),
 array([-2.62634254e-04, -9.13053192e-05,  1.59884975e-04]),
 array([-0.45960029, -0.80138042, -0.00219163]))

In [95]:
ro = 1/np.linalg.norm(a3)
r3 = ro*a3
X0 = ro*ro*np.dot(a1, a3);
Y0 = ro*ro*np.dot(a2, a3);
alpha = ro*ro*np.linalg.norm(np.cross(a1, a3));
beta = ro*ro*np.linalg.norm(np.cross(a2, a3));

In [96]:
ro,r3,X0,Y0,alpha,beta

(3117.7526826214853,
 array([-0.81882865, -0.2846674 ,  0.49848181]),
 954.2410194116792,
 72.0558592078065,
 694.5308420735313,
 162.48422622005043)

In [98]:
cosTheta = -np.dot(np.cross(a1,a3),np.cross(a2,a3))/(np.linalg.norm(np.cross(a1,a3))*np.linalg.norm(np.cross(a2,a3)))
sinTheta = (1-cosTheta**2)**(1/2)
r1 = np.cross(a2,a3)/np.linalg.norm(np.cross(a2,a3))
r2 = np.cross(r3,r1)

In [99]:
cosTheta,sinTheta,r1,r2

(-0.8627149906362609,
 0.5056904635559941,
 array([-0.16262522,  0.94783826,  0.27414535]),
 array([-0.55052038,  0.14341236, -0.82241122]))

### After Calculation of parameters, We double check by calculating 2d points from 3D and compare. 